In [1]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import random
import nnfs
from nnfs.datasets import spiral_data
import math

nnfs.init()

In [2]:
# implementing loss

In [3]:
class Layer_Dense:
    def __init__(self, n_inputs, n_neurons): # pass in size of inputs (size of a single sample) and number of neurons
        # note 28:17 we won't need to take the transpose when we do the forward pass
        self.weights = 0.10 * np.random.randn(n_inputs, n_neurons) # shape is n_inputs x n_neurons
        self.biases = np.zeros((1, n_neurons)) 
    def forward(self, inputs):
        self.output = np.dot(inputs, self.weights) + self.biases

class Activation_ReLU:
    def forward(self, inputs):
        self.output = np.maximum(0, inputs)
        
class Activation_Softmax:
    def forward(self, inputs):
        exp_values = np.exp(inputs - np.max(inputs, axis=1, keepdims = True))  #inputs is a batch of outputs
        probabilities = exp_values / np.sum(exp_values, axis = 1, keepdims = True) # sum each row
        self.output = probabilities
        
class Loss:
    def calculate(self, output, y): #output from the model, intended target values
        sample_losses = self.forward(output, y)
        data_loss = np.mean(sample_losses) # batch loss
        return data_loss
    
class Loss_CategoricalCrossEntropy(Loss):
    def forward(self, y_pred, y_true):
        samples = len(y_pred)
        y_pred_clipped = np.clip(y_pred, 1e-7, 1-1e-7)
        
        if len(y_true.shape) == 1: # this means they passed scalar values, not one hot encoding
            correct_confidences = y_pred_clipped[range(samples), y_true]
        elif len(y_true.shape) == 2: # one hot encoded vectors are being passed
            correct_confidences = np.sum(y_pred_clipped*y_true, axis=1)
            
        negative_log_likelihoods = -np.log(correct_confidences)
        return negative_log_likelihoods

In [5]:
X, y = spiral_data(samples=100, classes=3)

dense1 = Layer_Dense(2,3)
activation1 = Activation_ReLU()

dense2 = Layer_Dense(3,3)
activation2 = Activation_Softmax()

dense1.forward(X)
activation1.forward(dense1.output)

dense2.forward(activation1.output)
activation2.forward(dense2.output)

print(activation2.output[:5]) # first 5 of 300 outputs. this is a batch of 300.

loss_function = Loss_CategoricalCrossEntropy()
loss = loss_function.calculate(activation2.output, y)

print("loss", loss)

[[0.33333334 0.33333334 0.33333334]
 [0.33335656 0.33332282 0.33332065]
 [0.3333794  0.33331248 0.3333082 ]
 [0.33340728 0.3332996  0.3332931 ]
 [0.3334332  0.33328772 0.33327907]]
loss 1.0982093


In [ ]:
softmax_output = [0.7, 0.1, 0.2]

target_class = 0  # at index 0 it's "hot" otherwise it's n class length, which is 3
target_output = [1, 0, 0]

loss = -(math.log(softmax_output[0]) * target_output[0] +
        math.log(softmax_output[1]) * target_output[1] +
        math.log(softmax_output[2]) * target_output[2])
# equivalent to:
loss = -(math.log(softmax_output[0]))
print(loss)

# loss is inversely proportional to confidence.